In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf # pip install tensorflow-macos
import os
tf.random.set_seed(1)

# The typical architecture of a Recurrent Neural Network (RNN)
- The premise of an RNN is simple: use information from the past to help you with the future (this is where the term recurrent comes from). In other words, take an input (X) and compute an output (y) based on all previous inputs.

When an RNN looks at a sequence of text (already in numerical form), the patterns it learns are continually updated based on the order of the sequence.

For a simple example, take two sentences:

1. Massive earthquake last week, no?
2. No massive earthquake last week.

Both contain exactly the same words but have different meaning. The order of the words determines the meaning (one could argue punctuation marks also dictate the meaning but for simplicity sake, let's stay focused on the words).

### actual architecture
1. input layer
2. text vectorization layer
3. embedding
4. RNN cell(s)
5. Hidden activation
6. pooling layer (sometimes, usually for Conv1D models)
7. fully connected layer
8. output layer



# Preparing a notebook for our first NLP with TensorFlow project
[data from kaggle](https://www.kaggle.com/competitions/nlp-getting-started/leaderboard)

In [2]:
from helper_functions import create_tensorboard_callback, unzip_data, plot_loss_curves, compare_historys

In [3]:
zip_path = "nlp_getting_started.zip"
if not os.path.isfile(zip_path):
    os.chdir("data")
unzip_data(zip_path)

# Becoming one with the data and visualizing a text dataset

In [12]:
# we can use pandas because the data isn't too big
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# shuffle training data
train_df = train_df.sample(frac=1, random_state=1)

In [13]:
train_df.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [14]:
train_df.iloc[0]["text"]

'Goulburn man Henry Van Bilsen missing: Emergency services are searching for a Goulburn man who disappeared from his\x89Û_ http://t.co/z99pKJzTRp'

In [15]:
# test data frame looks the same but without targets
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [16]:
train_df["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [17]:
print(len(train_df), len(test_df))

7613 3263


In [18]:
# let's visualize some random samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
Text:
@Haley_Whaley Hailstorm Hey There is a Secret Trick to get 375.000 Gems Clash ofClans check them now on my Profile

---

Target: 0 (not real disaster)
Text:
I'm mentally preparing myself for a bomb ass school year if it's not I'm burning buildings ??

---

Target: 0 (not real disaster)
Text:
What's wrong with just a lil smoke and good conversation ????

---

Target: 0 (not real disaster)
Text:
@wrongdejavu I'm traumatised

---

Target: 1 (real disaster)
Text:
The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/aueZxZA5ak

---



# Splitting data into training and validation sets

In [20]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                            train_df["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=1)

In [22]:
print(len(train_sentences), len(train_labels), len(val_sentences), len(val_labels))

6851 6851 762 762


In [26]:
train_sentences[:-10]

array(['Rly tragedy in MP: Some live to recount horror: \x89ÛÏWhen I saw coaches of my train plunging into water I called my daughters and said t...',
       'A river of lava in the sky this evening! It was indeed a beautiful sunset sky tonight. (8-4-15) http://t.co/17EGMlNi80',
       'Los Angeles Times: Arson suspect linked to 30 fires caught in Northern ... - http://t.co/xwMs1AWW8m #NewsInTweets http://t.co/TE2YeRugsi',
       ...,
       "When you lowkey already know you're gonna drown in school this year :) http://t.co/aCMrm833zq",
       '@JamesMelville Some old testimony of weapons used to promote conflicts\nTactics - corruption &amp; infiltration of groups\nhttps://t.co/cyU8zxw1oH',
       "Just got evacuated from the movie theatre for an emergency. Saw people running from another they're."],
      dtype=object)

In [27]:
train_labels[:-10]

array([1, 0, 1, ..., 0, 0, 1])

# Converting text data to numbers using tokenisation and embeddings (overview)
### Tokenization
- A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:
1. Using word-level tokenization with the sentence "I love TensorFlow" might result in "I" being 0, "love" being 1 and "TensorFlow" being 2. In this case, every word in a sequence considered a single token.
2. Character-level tokenization, such as converting the letters A-Z to values 1-26. In this case, every character in a sequence considered a single token.
3. Sub-word tokenization is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple tokens.


### Embeddings
- An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. It's important to note here, the size of the feature vector is tuneable. There are two ways to use embeddings:
1. Create your own embedding - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as tf.keras.layers.Embedding) and an embedding representation will be learned during model training.
2. Reuse a pre-learned embedding - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

# Setting up a TensorFlow TextVectorization layer to convert text to numbers

In [30]:
from tensorflow.keras.layers import TextVectorization

In [33]:
# these are the default parameters. this cell is just for demonstration
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [32]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [34]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

# Mapping the TextVectorization layer to text data and turning it into numbers

In [35]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

2023-05-11 11:55:12.734364: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [36]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[319,   3, 199,   4,  13, 699,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [38]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
the sunset boys wreck my bed   original 1979 usa gimp label  vinyl 7' 45  newave http://t.co/X0QLgwoyMT http://t.co/hQNx8qMeG3      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 1979, 1725,  172,   13, 1092, 1620, 6329,  832,    1, 5244,
        1949,  694, 3195,    1]])>

In [39]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

# UNK is unknown token which replaces uncommon words. increasing max tokens will reduce it's popularity

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['palmer', 'palm', 'palinfoen', 'palestinian\x89Û', 'paleface']
